In [0]:
# Databricks notebook source
import yaml
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Initialize Spark with Delta + UC support
builder = (
    SparkSession.builder.appName("extract-weather-raw")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# 1️⃣ Load config
with open("/Workspace/Repos/ziaarzoo21@gmail.com/spark-etl/config/config.yaml") as f:
    cfg = yaml.safe_load(f)

# 2️⃣ Ensure catalog + schema exist (safe check)
spark.sql(f"CREATE CATALOG IF NOT EXISTS {cfg['catalog_name']}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {cfg['catalog_name']}.{cfg['schema_name']}")

# 3️⃣ Run Extract Layer
from src.extract.save_raw import run as extract_run
table_written = extract_run(cfg, spark)

# 4️⃣ Preview written data
df = spark.table(table_written)
display(df.limit(10))

# 5️⃣ Show storage location (to confirm Unity Catalog linked ADLS)
spark.sql(f"DESCRIBE DETAIL {table_written}").select("location").display()
